In [1]:
import tensorflow
from tensorflow.keras.models import load_model
import pickle
import pandas as pd


In [2]:
model=load_model('model.weights.h5')
#with open('le.pkl','rb') as file:
    #le=pickle.load(file)
le=pickle.load(open('le.sav','rb'))
   

In [3]:
#with open('ohe.pkl','rb')as file:
    #ohe=pickle.load(file)
#with open('scaler.pkl','rb')as file:
    #scaler=pickle.load(file)
ohe=pickle.load(open('ohe.sav','rb'))
scaler=pickle.load(open('scaler.sav','rb'))


In [4]:
input_data={
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [5]:
df=pd.DataFrame(input_data,index=[0])

In [6]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [7]:
df['Gender']=le.transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [8]:
encoded=ohe.transform(df[['Geography']]).toarray()

In [9]:
encoded

array([[1., 0., 0.]])

In [10]:
dff=pd.DataFrame(encoded,columns=ohe.get_feature_names_out(['Geography']))
dff

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [11]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [12]:
df=df.drop('Geography',axis=1)

In [13]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,1,40,3,60000,2,1,1,50000


In [14]:
encoded_df=pd.concat([df,dff],axis=1)
encoded_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [15]:
scalers=scaler.transform(encoded_df)

In [16]:
scalers

array([[-0.47252305,  0.91407657,  0.0906231 , -0.70903082, -0.27592043,
         0.79187296,  0.63599873,  0.97160303, -0.84394851,  0.98728088,
        -0.58166121, -0.56318378]])

In [17]:
prediction=model.predict(scalers)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


array([[0.03930333]], dtype=float32)

In [18]:
prediction_prob=prediction[0][0]
prediction_prob

0.039303325

In [19]:
if prediction_prob>0.5:
    print('Customer is likely to churn')
else:
    print('Customer is not likely to churn')

Customer is not likely to churn
